In [0]:

%pip install scikit-image
%pip install cudf-cu11 --extra-index-url https://pypi.nvidia.com 
%pip install cuml-cu11 --extra-index-url https://pypi.nvidia.com
%pip install dask-ml
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached https://pypi.nvidia.com/cudf-cu11/cudf_cu11-23.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (499.6 MB)
  Using cached nvtx-0.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (553 kB)
  Using cached cupy_cuda11x-12.2.0-cp310-cp310-manylinux2014_x86_64.whl (89.6 MB)
  Using cached numba-0.57.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.6 MB)
  Using cached cuda_python-11.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
  Using cached https://pypi.nvidia.com/ptxcompiler-cu11/ptxcompiler_cu11-0.7.0.post1-cp310-cp310-manylinux_2

In [0]:
import numpy as np  
import pandas as pd  
import xgboost as xgb  
from sklearn.compose import ColumnTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline  
from sklearn.preprocessing import StandardScaler  
from skimage.feature import hog 
import cudf

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x] = len(encode_dict)
    return encode_dict[x]

df = pd.read_parquet('/dbfs/FileStore/tables/data_optimized_proc.parquet')
df = df.dropna(subset=["IssueType", "screenshot"])
df["CAT"] = df["IssueType"].map(lambda x: encode_cat(x))
print(df.groupby(["CAT", "IssueType"])["CAT"].count())
print(df.groupby(["CAT", "IssueType"])["CAT"].count() / df.shape[0])
df = cudf.from_pandas(df)
df.head()

In [0]:
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(
    df, train_size=0.8, random_state=42, stratify=df.CAT
)
train_dataset = train_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)
print("- TRAIN -\n")
print(train_dataset.groupby(["CAT", "IssueType"])["CAT"].count())
print("-----------------")
print("- TEST -\n")
print(test_dataset.groupby(["CAT", "IssueType"])["CAT"].count())

In [0]:
import cuml
def get_params():
    learning_rate = st.uniform(0.05, 0.4)
    max_depth = [10, 20, 50, 100, 200, 300, 500]
    n_estimators = [5, 10, 25, 50, 75, 100, 200, 400]
    reg_lambda = [0.1, 0.3, 1]
    colsample_bytree = st.beta(10, 1)
    gamma = st.uniform(0, 10)
    subsample = [0.6, 0.8, 1.0]
    reg_alpha = st.expon(0, 50)
    min_child_weight = [1, 5, 10]
    tree_method = 'gpu_hist'
    # Create the random grid
    return {
        "vectorizer__analyzer": ["word"],
        "vectorizer__stop_words": ["english"],
        "vectorizer__strip_accents": ["ascii"],
        "vectorizer__ngram_range": [(1, 1), (1, 2), (2, 2)],
        "classifier__n_estimators": n_estimators,
        "classifier__learning_rate": learning_rate,
        "classifier__reg_lambda": reg_lambda,
        "classifier__colsample_bytree": colsample_bytree,
        "classifier__gamma": gamma,
        "classifier__subsample": subsample,
        "classifier__reg_alpha": reg_alpha,
        "classifier__max_depth": max_depth,
        "classifier__min_child_weight": min_child_weight,
        "classifier__tree_method": tree_method
    }
print('RAPIDS',cuml.__version__)

In [0]:
import numpy as np  
import pandas as pd  
from xgboost import XGBClassifier
import xgboost
from sklearn.compose import ColumnTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline  
from sklearn.preprocessing import StandardScaler  
from skimage.feature import hog  
import scipy.stats as st
import mlflow
from mlflow.tracking import MlflowClient
import matplotlib.pyplot as plt
from mlflow.utils.environment import _mlflow_conda_env
from mlflow.models.signature import infer_signature
import matplotlib
import io
from PIL import Image
import numpy as np
import seaborn as sns
import sklearn
from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split,
    StratifiedKFold,
)
from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_recall_fscore_support,
    f1_score,
    recall_score,
    precision_score,
)
import dask_ml.model_selection as dcv
import cuml
from cuml.feature_extraction.text import TfidfVectorizer
import xgboost as xgb

"""
# Load image data and extract HOG features  
hog_features = np.array([hog(np.array(df.loc[i, "screenshot"])) for i in range(len(df))])  
  
# Load text data and create feature matrix  
vectorizer = CountVectorizer()  
text_features = vectorizer.fit_transform(df["StepDescription"]).toarray()  
  
# Combine image and text features  
features = np.hstack((hog_features, text_features))  
  
# Create a Scikit-learn pipeline  
preprocessor = ColumnTransformer(transformers=[  
        ("scaler", StandardScaler(), slice(0, len(hog_features))),  
        ("nothing", "passthrough", slice(len(hog_features), None))  
    ])  

model = Pipeline(steps=[  
        ("preprocessor", preprocessor),  
        ("classifier", xgb.XGBClassifier())  
    ])  
  
# Train the model  
X = features  
y = image_data["label"]  
model.fit(X, y)  
  
# Use the model to make predictions  
X_new = features[:10]  # Example input data  
y_pred = model.predict(X_new)  

"""


def train_tree_model(
    train_data: pd.DataFrame, cv_params: dict, n_cv_folds: int, n_iter: int
):
    """
    :param train_data: Preprocessed dataframe that contains both the question and category
    """
   # hog_features = np.array([hog(np.array(Image.open(io.BytesIO(train_data.loc[i, "screenshot"]))), channel_axis=-1) for i in range(len(train_data))]) 

    vectorizer = TfidfVectorizer()  
    text_features = vectorizer.fit_transform(df["StepDescription"]).toarray()  

    #X = np.hstack((hog_features, text_features)) 
    X = train_data.StepDescription#, 'image': hog_features}
    Y = train_data["CAT"]

    params = {
        'learning_rate': 0.1,
        'colsample_bytree' : 0.3,
        'max_depth': 5,
        'objective': 'reg:linear',
        'n_estimators':10,
        'alpha' : 10,
        'silent': True,
        'verbose_eval': True,
        'tree_method':'gpu_hist',
    }
    dtrain = xgb.DMatrix(text_features, label=train_data["CAT"])
    classifier = xgb.train(params, dtrain, num_boost_rounds=100, evals=[(dtrain, 'train')])

   # preprocessor = ColumnTransformer(transformers=[  
   #     ("scaler", StandardScaler(), slice(0, len(hog_features))),  
   #     ("nothing", "passthrough", slice(len(hog_features), None))  
   # ])  

   # preprocessor = ColumnTransformer(transformers=[  
    #    ("scaler", StandardScaler(), slice(0, len(hog_features))),  
    #    ("tfidf", TfidfVectorizer(), "text")  
   # ]) 
    
   # classifier = Pipeline(
    #    steps=[
     #       ("vectorizer", TfidfVectorizer()),
           # ("preprocessor", TfidfVectorizer()),
      #      ("classifier", XGBClassifier(random_state=10)),
      #  ],
      #  verbose=True,
    #)
   # classifier_random = dcv.RandomizedSearchCV(
   #     classifier,
   #     param_distributions=cv_params,
   #     n_iter=n_iter,
   #     verbose=0,
   #     scoring="accuracy",
   #     random_state=42,
   #     refit=True,
   #     n_jobs=-1,
   #     cv=StratifiedKFold(n_splits=n_cv_folds, shuffle=True, random_state=42),
   # )
   # classifier_random.fit(X, Y)
    return classifier

In [0]:
class SKLearnEstimatorWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, estimator, mapping_dict):
        self.estimator = estimator
        self.mapping_dict = mapping_dict

    def predict(self, context, model_input):
        #image_features = np.array([hog(np.array(Image.open(io.BytesIO(model_input.loc[i, "screenshot"]))), channel_axis=-1) for i in range(len(model_input))]) 

        model_input = model_input["StepDescription"]#, 'image': image_features}

        outputs = self.estimator.predict(model_input)
        ret_d = {"class": np.array([self.mapping_dict[str(x)] for x in outputs])}
        return ret_d


def log_estimator_model(estimator, encode_dict):
    """
    :param estimator: An estimator that takes as input text and outputs a category (for the question)
    :param encode_dict: A dictionary which encodes a labelled category (such as 'Guest Rooms') to a numeric category
    """
    mapping_dict = {str(v): k for k, v in encode_dict.items()}
    wrapped_estimator = SKLearnEstimatorWrapper(estimator, mapping_dict)
    input_example = {
        "text": np.array(
            test_dataset.loc[0, 'StepDescription']
        )
        #'image': hog(np.array(Image.open(io.BytesIO(test_dataset.loc[0, "screenshot"]))), channel_axis=-1)
    }
    signature = infer_signature(
        pd.DataFrame(input_example),
        wrapped_estimator.predict(None, pd.DataFrame(input_example)),
    )
    # Specify the additional dependencies
    conda_env = _mlflow_conda_env(
        additional_conda_deps=None,
        additional_pip_deps=[
            "xgboost=={}".format(xgboost.__version__),
            "sklearn=={}".format(sklearn.__version__),
        ],
        additional_conda_channels=None,
    )
    # Log model so the model artifacts can have the correct conda env, python model, and signature (data formats for input & output)
    return mlflow.pyfunc.log_model(
        "Bluecumber_test_analysis",
        python_model=wrapped_estimator,
        conda_env=conda_env,
        signature=signature,
        input_example=input_example,
    )

In [0]:
client = MlflowClient()
n_cv_folds = 3


def init_experiment(experiment_name):
    # fetch existing experiments, if it doesn't exist create it and set it
    existing_experiments = client.search_experiments()
    if experiment_name in [exp.name for exp in existing_experiments]:
        return mlflow.set_experiment(experiment_name)
    else:
        return mlflow.create_experiment(experiment_name)

In [0]:
mlflow.end_run()
#experiment_id = init_experiment(
  #  "/Users/Chris.Jose@cvent.com/test_result_analysis/experiments/#xgboost_test_result_analysis_1")
#run = mlflow.start_run(run_name="xgboost_tfidf_CV", #experiment_id=experiment_id)

experiment_name = "/Users/Chris.Jose@cvent.com/test_result_analysis/experiments/#xgboost_test_result_analysis_1"
mlflow.set_experiment(experiment_name)  
  
# Get the experiment ID  
experiment = mlflow.get_experiment_by_name(experiment_name)  
experiment_id = experiment.experiment_id  
  
# Start a new MLflow run  
run = mlflow.start_run(run_name="xgboost_tfidf_CV", experiment_id=experiment_id)


mlflow.sklearn.autolog(log_models=False)
mlflow.log_param("n_cv_folds", 3)
classifier_random = train_tree_model(
    train_dataset, get_params(), n_cv_folds, n_iter=30000
)
print(classifier_random.best_score_)
#test_image_features = np.array([hog(np.array(Image.open(io.BytesIO(test_dataset.loc[i, "screenshot"]))), channel_axis=-1) for i in range(len(test_dataset))]) 

#model_input = {'text': test_dataset["StepDescription"], 'image': test_image_features}
X_test = test_dataset["StepDescription"]
Y_test = test_dataset["CAT"]
mlflow.log_metric(
    "testing_accuracy_score", classifier_random.best_estimator_.score(X_test, Y_test)
)
estimator = classifier_random.best_estimator_

In [0]:
y_pred = classifier_random.best_estimator_.predict(X_test)
y_true = Y_test

In [0]:
font = {"family": "normal", "size": 15}

matplotlib.rc("font", **font)
cm = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots()
sns.heatmap(
    cm,
    annot=True,
    annot_kws={"size": 12},
    ax=ax,
    cmap="Blues",
    fmt="g",
    xticklabels=list(encode_dict.keys()),
    yticklabels=list(encode_dict.keys()),
)
plt.xticks(rotation=45, horizontalalignment="right")
fig.set_size_inches(12, 12)
plt.title("Confusion Matrix - Test Set", fontdict={"family": "normal", "size": 21})
img_buf = io.BytesIO()
plt.ylabel("True label", fontdict={"family": "normal", "size": 21})
plt.xlabel("Predicted label", fontdict={"family": "normal", "size": 21})
plt.show()
fig.savefig(img_buf, dpi=fig.dpi, bbox_inches="tight")

In [0]:
font = {"family": "normal", "size": 15}

matplotlib.rc("font", **font)
cm = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots()
sns.heatmap(
    cm / cm.sum(axis=1).reshape(-1, 1),
    annot=True,
    annot_kws={"size": 12},
    ax=ax,
    cmap="Blues",
    fmt=".2%",
    xticklabels=list(encode_dict.keys()),
    yticklabels=list(encode_dict.keys()),
)
plt.xticks(rotation=45, horizontalalignment="right")
fig.set_size_inches(12, 12)
plt.title(
    "Confusion Matrix - Test Set Percentage", fontdict={"family": "normal", "size": 21}
)
plt.ylabel("True label", fontdict={"family": "normal", "size": 21})
plt.xlabel("Predicted label", fontdict={"family": "normal", "size": 21})
img_buf2 = io.BytesIO()
plt.show()
fig.savefig(img_buf2, dpi=fig.dpi, bbox_inches="tight")